# Titanic Dataset – Data Cleaning & Feature Engineering

Este proyecto realiza un proceso completo de **limpieza, transformación y análisis de datos**
sobre el dataset del Titanic utilizando **Python y pandas**.

El objetivo es preparar los datos para un análisis posterior, aplicando técnicas habituales
de data cleaning, feature engineering y lógica de negocio.


## Objetivos del proyecto

- Analizar y detectar valores nulos
- Aplicar diferentes estrategias de imputación
- Limpiar columnas de texto mediante expresiones regulares
- Crear nuevas variables (feature engineering)
- Realizar análisis numérico y rankings
- Calcular métricas personalizadas por pasajero
